In [ ]:
%pylab

In [ ]:
import cPickle as pickle
import gzip

In [ ]:
#data = load('PMPulse_1_6_Vmon-1.pkl')
#pickle.dump(data,gzip.open('PMPulse_1_6_Vmon-1.pklz','wb'))

In [ ]:
# Load Data
data = pickle.load(gzip.open('PMPulse_1_6_Vmon-1.pklz','rb'))
dt = 1/20000. #20 kHz sampling rate

In [ ]:
print data.shape
#the first dimension are sweeps with different strengths of depolarization currents (50pA steps), second dimension is time
#stim_current = ???

# select the voltage trace of the last sweep
V = data[-1]

## Part I

__1)__ 
- How long is the complete voltage trace in seconds?
- How long was the stimulation?
- Plot the  complete voltage trace
- Plot only the period of the stimulation
- Set the range of the yaxis to (-75mV, 20mV).

__2)__
Using the voltage trace, determine the times at which the cell fired action potentials.
- How many action potentials did you detect?
- What is the average firing rate (i.e. action potentials per second) of this neuron?

__3)__ Spike Triggered Average (STA)

Now determine the average waveform of an action potential. To this end you need to get  all AP onset times to obtain a cut-out of about +/- 10 ms around the onset time and average across all the cutouts.

- Add to the plot the average waveform ± 1 standard deviation across all APs.

__4)__

What is the mean and variance of the membrane potential distribution when excluding the action potential cutouts?
- n the period without stimulation
- in the period during stimulation

__5)__

The standard integrate-and-fire (I&F) model of a neuron assumes a fixed voltage threshold for initiating a spike. Can you determine a fixed voltage threshold that determines the onset of each spike? Repeat exercise 3 but now with a longer cut-out before the spike detection of 500ms.

Plot the mean action potential in red and in addition plot two functions as mean ± standard deviation.